In [1]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.cuda()

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
import pandas as pd
import numpy as np

In [4]:
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
!wget https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv

--2023-12-11 00:31:58--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSentences.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1290254 (1.2M) [text/plain]
Saving to: ‘datasetSentences.csv’

datasetSentences.cs 100%[===================>]   1.23M  --.-KB/s    in 0.06s   

2023-12-11 00:31:58 (22.0 MB/s) - ‘datasetSentences.csv’ saved [1290254/1290254]

--2023-12-11 00:32:00--  https://gist.githubusercontent.com/kvyaswanth/29d2858f54642494aa86d1bdec0a3a3b/raw/b3697e379551fe523f05f40adc841a48dbbb120a/datasetSplit.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubuserc

In [5]:
df = pd.read_csv('datasetSentences.csv')
labels = pd.read_csv('datasetSplit.csv')

In [6]:
df

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,Emerges as something rare ; an issue movie tha...
...,...,...
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...


In [7]:
def inp_ids(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['input_ids']

def token_types(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['token_type_ids']

def attn_mask(text):
  return tokenizer(text, return_tensors="pt", padding=True, truncation=True)['attention_mask']

In [8]:
tokenizer('text', return_tensors="pt", padding=True, truncation=True)

{'input_ids': tensor([[1499,    1]]), 'attention_mask': tensor([[1, 1]])}

In [9]:
df['input_ids'] = df['sentence'].apply(inp_ids)
# df['token_type_ids'] = df['sentence'].apply(token_types)
df['attention_mask'] = df['sentence'].apply(attn_mask)

In [10]:
df

,sentence_index,sentence,input_ids,attention_mask
0,1,The Rock is destined to be the 21st Century 's...,"[[tensor(37), tensor(3120), tensor(19), tensor...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
1,2,The gorgeously elaborate continuation of `` Th...,"[[tensor(37), tensor(5296), tensor(120), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,3,Effective but too-tepid biopic,"[[tensor(18652), tensor(68), tensor(396), tens...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,4,If you sometimes like to go to the movies to h...,"[[tensor(156), tensor(25), tensor(1664), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,5,Emerges as something rare ; an issue movie tha...,"[[tensor(25579), tensor(15), tensor(7), tensor...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
...,...,...,...,...
11850,11851,A real snooze .,"[[tensor(71), tensor(490), tensor(3), tensor(7...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11851,11852,No surprises .,"[[tensor(465), tensor(17844), tensor(3), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11852,11853,We 've seen the hippie-turned-yuppie plot befo...,"[[tensor(101), tensor(3), tensor(31), tensor(1...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
11853,11854,Her fans walked out muttering words like `` ho...,"[[tensor(1347), tensor(2675), tensor(3), tenso...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


In [11]:
def extract_embeddings(row):
    inputs = {
        'input_ids': row['input_ids'].cuda(),
#         'token_type_ids': row['token_type_ids'].cuda(),
        'attention_mask': row['attention_mask'].cuda()
    }

    with torch.no_grad():
        outputs = model.encoder(**inputs)
        embeddings = outputs.last_hidden_state
        return embeddings.cpu().numpy()

In [12]:
df['embeddings'] = df[['input_ids','attention_mask']].apply(extract_embeddings, axis=1)

In [13]:
for i in df['embeddings']:
    print(i.shape)

(1, 60, 512)
(1, 65, 512)
(1, 10, 512)
(1, 26, 512)
(1, 34, 512)
(1, 30, 512)
(1, 12, 512)
(1, 23, 512)
(1, 33, 512)
(1, 11, 512)
(1, 18, 512)
(1, 19, 512)
(1, 22, 512)
(1, 40, 512)
(1, 20, 512)
(1, 18, 512)
(1, 30, 512)
(1, 13, 512)
(1, 10, 512)
(1, 32, 512)
(1, 15, 512)
(1, 17, 512)
(1, 50, 512)
(1, 25, 512)
(1, 32, 512)
(1, 23, 512)
(1, 19, 512)
(1, 22, 512)
(1, 26, 512)
(1, 31, 512)
(1, 35, 512)
(1, 22, 512)
(1, 31, 512)
(1, 10, 512)
(1, 49, 512)
(1, 47, 512)
(1, 22, 512)
(1, 36, 512)
(1, 34, 512)
(1, 60, 512)
(1, 32, 512)
(1, 18, 512)
(1, 28, 512)
(1, 30, 512)
(1, 20, 512)
(1, 26, 512)
(1, 22, 512)
(1, 40, 512)
(1, 18, 512)
(1, 33, 512)
(1, 16, 512)
(1, 54, 512)
(1, 13, 512)
(1, 19, 512)
(1, 24, 512)
(1, 16, 512)
(1, 33, 512)
(1, 48, 512)
(1, 28, 512)
(1, 21, 512)
(1, 60, 512)
(1, 26, 512)
(1, 10, 512)
(1, 56, 512)
(1, 15, 512)
(1, 19, 512)
(1, 35, 512)
(1, 31, 512)
(1, 36, 512)
(1, 18, 512)
(1, 37, 512)
(1, 25, 512)
(1, 33, 512)
(1, 34, 512)
(1, 11, 512)
(1, 32, 512)
(1, 29, 512)

In [14]:
# Find the maximum sequence length
max_length = max([tensor.shape[1] for tensor in df.embeddings])

In [15]:
def pad_tensor(tensor, max_length):
    padding_length = max_length - tensor.shape[1]
    if padding_length > 0:
        pad =(0, 0, 0, padding_length)
        tensor = torch.nn.functional.pad(tensor, pad, "constant", 0)
    return tensor


In [16]:
# Pad each tensor in the DataFrame
X = [pad_tensor(torch.tensor(tensor), max_length) for tensor in df.embeddings]

In [17]:
# Now you can stack them
X = torch.vstack(X)

In [18]:
X = [tensor.mean(dim=0) for tensor in X]

In [19]:
X = torch.vstack(X)

In [20]:
X.shape

torch.Size([11855, 512])

In [21]:
labels

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2
...,...,...
11850,11851,1
11851,11852,1
11852,11853,1
11853,11854,1


In [22]:
y = labels['splitset_label'].values

In [23]:
labels['splitset_label'].value_counts()

splitset_label
1    8544
2    2210
3    1101
Name: count, dtype: int64

In [24]:
from imblearn.over_sampling import SMOTE

# Instantiate SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to the data
X_resampled, y_resampled = smote.fit_resample(X, y)


In [25]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [26]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier with 100 trees (you can adjust this parameter)
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [27]:
# Make predictions on the test set
predictions = clf.predict(X_test)

# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

accuracy = accuracy_score(y_test, predictions)
report = classification_report(y_test, predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)


Accuracy: 0.9233469865418373
Classification Report:
               precision    recall  f1-score   support

           1       0.87      0.91      0.89      1744
           2       0.92      0.89      0.90      1705
           3       0.98      0.97      0.98      1678

    accuracy                           0.92      5127
   macro avg       0.93      0.92      0.92      5127
weighted avg       0.92      0.92      0.92      5127

